In [2]:
import numpy as np
mu, sigma = 0, 0.1 # mean and standard deviation
s = np.random.normal(mu, sigma, 1000)

In [23]:
#generate traing data 
def fx(x,e):
    num=0
    y=[]
    while num<len(x):
        z=int(x[num])
        yy=z**3+e[num]
        #print(yy)
        num+=1
        y.append(yy)
    return y

In [24]:
import random
x=[]
for i in range(1000):
    z=random.uniform(1, 10)
    x.append(z)
y=fx(x,s)

In [26]:
y=np.array(y)
x=np.array(x)

In [30]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model


# Define inputs with predefined shape
inputs = Input(shape=(1,))

# Build network with some predefined architecture
output1 = Dense(10,activation='relu')(inputs)
output2 = Dense(10,activation='relu')(output1)

# Predict the target variable with a single Dense output
regression_output = Dense(2)(output2)

# Construct model
model = Model(inputs=inputs, outputs=regression_output)

In [31]:
import keras.backend as K
import numpy as np


def gaussian_nll(ytrue, ypreds):
    """Keras implmementation of multivariate Gaussian negative loglikelihood loss function. 
    This implementation implies diagonal covariance matrix.
    
    Parameters
    ----------
    ytrue: tf.tensor of shape [n_samples, n_dims]
        ground truth values
    ypreds: tf.tensor of shape [n_samples, n_dims*2]
        predicted mu and logsigma values (e.g. by your neural network)
        
    Returns
    -------
    neg_log_likelihood: float
        negative loglikelihood averaged over samples
        
    This loss can then be used as a target loss for any keras model, e.g.:
        model.compile(loss=gaussian_nll, optimizer='Adam') 
    
    """
    
    n_dims = int(int(ypreds.shape[1])/2)
    mu = ypreds[:, 0:n_dims]
    logsigma = ypreds[:, n_dims:]
    
    mse = -0.5*K.sum(K.square((ytrue-mu)/K.exp(logsigma)),axis=1)
    sigma_trace = -K.sum(logsigma, axis=1)
    log2pi = -0.5*n_dims*np.log(2*np.pi)
    
    log_likelihood = mse+sigma_trace+log2pi

    return K.mean(-log_likelihood)

Using TensorFlow backend.


In [34]:
model.compile(optimizer='adam', loss=gaussian_nll)
model_fit = model.fit(x, y, 
                      batch_size=10, epochs=40
                   )

Epoch 1/40
100/100 [==============================] - 0s 721us/step - loss: 6.6725
Epoch 2/40
100/100 [==============================] - 0s 792us/step - loss: 6.0436
Epoch 3/40
100/100 [==============================] - 0s 839us/step - loss: 6.0315
Epoch 4/40
100/100 [==============================] - 0s 797us/step - loss: 6.0188
Epoch 5/40
100/100 [==============================] - 0s 782us/step - loss: 6.0061
Epoch 6/40
100/100 [==============================] - 0s 759us/step - loss: 5.9936
Epoch 7/40
100/100 [==============================] - 0s 754us/step - loss: 5.9828
Epoch 8/40
100/100 [==============================] - 0s 763us/step - loss: 5.9711
Epoch 9/40
100/100 [==============================] - 0s 752us/step - loss: 5.9620
Epoch 10/40
100/100 [==============================] - 0s 776us/step - loss: 5.9521
Epoch 11/40
100/100 [==============================] - 0s 747us/step - loss: 5.9442
Epoch 12/40
100/100 [==============================] - 0s 743us/step - loss: 5.9357
E

In [35]:
model.predict(x)

array([[46.958523 ,  7.2199183],
       [18.329288 ,  3.204102 ],
       [28.257504 ,  4.601701 ],
       ...,
       [ 8.7618065,  1.8527968],
       [46.45367  ,  7.1492367],
       [38.124897 ,  5.9831758]], dtype=float32)